# Preprocessing

This tutorial covers the preprocessing steps of TF-MInDi:
- Loading motif collections and annotations
- Extracting seqlets from contribution scores
- Calculating motif similarities
- Creating and saving tfmindi's custom {class}anndata.Anndata objects

## Data Requirements

To get started with `tfmindi`, we need three types of data:
- a motif database of known transcription factor motifs
- one hot encoded sequences for genomic regions of interest
- attribution scores representing nucleotide importances for these same regions from a trained neural network.  

`tfmindi` provides functionality to fetch SCENIC+ motif collections if you don't have your own available.  
Getting attribution scores and one-hot encoded regions is outside the scope of `tfmindi`. We recommend using CREsted for this (i.e. with [this function](https://crested.readthedocs.io/en/latest/api/tools/_autosummary/crested.tl.contribution_scores_specific.html)), yet any tool that can do this works fine (such as [tangermeme](https://tangermeme.readthedocs.io/en/latest/tutorials/Tutorial_A3_Deep_LIFT_SHAP.html)). So long as you can load these into python as numpy arrays.

## Fetching motif collections and annotations

On top of the contribution scores and sequences, we'll need motif collections to calculate similarity scores using {func}`~tfmindi.pp.calculate_motif_similarity` with the extracted seqlets.  
TF-MInDi provides functionality to fetch and load some of the default SCENIC+ motif databases.  
We can also fetch some motif annotations and motif-to-dbd mappings to visualize the seqlets later. 

In [1]:
import tfmindi as tm

import os
import re
import numpy as np
from tqdm import tqdm

DATA_DIR = "../../../../data/tfmindi/"  # change to your project's data directory

In [2]:
# fetch collection and annotations
motif_collection_dir = tm.fetch_motif_collection()
motif_annotations_file = tm.fetch_motif_annotations()

# to speed up all analyses, we can choose to only load a sampled subset of motifs. We have selected some and stored those names in a .txt file.
motif_samples_path = DATA_DIR + "sampled_motifs.txt"
with open(motif_samples_path) as f:
    motif_names = [line.strip() for line in f.readlines()]

# load them as dictionary of PPM matrices
motif_collection = tm.load_motif_collection(motif_collection_dir, motif_names=motif_names)
motif_annotations = tm.load_motif_annotations(motif_annotations_file)

# load motif to dna-binding domain (DBD) mapping
motif_to_db = tm.load_motif_to_dbd(motif_annotations)

## Extracting seqlets using tangermeme

We use `tangermeme` to extract seqlets (spans of nucleotides with high importance scores) from our nucleotide level contribution scores per cell type coming from a CREsted model.  
We calculated these scores for cell-type specific enhancer regions only so we will be able to link each region back to a specific cell type, but this is not required to run `tfmindi`.  

To extract seqlets, we use the {func}`~tfmindi.pp.extract_seqlets` function, which wraps `tangermeme`s [recursive_seqlets](https://tangermeme.readthedocs.io/en/latest/tutorials/Tutorial_A4_Seqlets.html#Recursive-Seqlets) functionality.  
The resulting seqlets will be scaled to a range of [-1,1] and sign corrected so the average contribution values are always positive. 

The expected input shape of the contribution scores and one hot-encoded regions is (N, 4, W).  

In [3]:
# extract_seqlets expects the inputs to be in shape (n, 4, region_width), so we concatenate the cell type specific contributions
CONTRIB_FOLDER = DATA_DIR + "modisco_results_ft_2000/"

contrib_list = []
oh_list = []
classes_list = []

# getting cell type names from file names (optional, not required to run tfmindi)
class_names = [
    re.match(r"(.+?)_oh\.npz$", f).group(1)  # type: ignore
    for f in os.listdir(CONTRIB_FOLDER)
    if f.endswith("_oh.npz")
]

for i, c in enumerate(tqdm(class_names)):
    contrib_list.append(np.load(os.path.join(CONTRIB_FOLDER, f"{c}_contrib.npz"))["arr_0"])
    oh_list.append(np.load(os.path.join(CONTRIB_FOLDER, f"{c}_oh.npz"))["arr_0"])
    classes_list.append(np.repeat(c, oh_list[i].shape[0]))

oh = np.concatenate(oh_list)
contrib = np.concatenate(contrib_list)
classes = np.concatenate(classes_list)  # not required to run tfmindi, but useful for interpretation later
region_id_to_ct_map = {
    i: str(c) for i, c in enumerate(classes)
}  # not required to run tfmindi, but useful for interpretation later

  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [00:29<00:00,  1.57s/it]


In [4]:
# extract seqlets from the contributions and one-hot encoded sequences
seqlets_df, seqlets_matrices = tm.pp.extract_seqlets(
    contrib=contrib,
    oh=oh,
    threshold=0.05,  # importance threshold for seqlet extraction. Lower = fewer seqlets.
    additional_flanks=3,  # flanking bases to include around seqlet
)

Processing seqlets: 100%|██████████| 679653/679653 [00:20<00:00, 33091.17it/s]


In [5]:
len(seqlets_matrices)

679653

In [6]:
seqlets_df.head(3)  # information on seqlet position and significance

,example_idx,start,end,attribution,p-value
0,18770,1052,1065,22.867273,2.963802e-15
1,19895,1103,1134,42.865262,2.971223e-15
2,19813,1045,1059,13.644972,4.303625e-15


In [7]:
seqlets_matrices[0].shape  # seqlets_matrices is a list with len(seqlets), that contains the scaled matrix per seqlet

(4, 13)

## Calculating motif similarity

Next, we calculate similarity scores using {func}`~tfmindi.pp.calculate_motif_similarity` between the extracted seqlets and the SCENIC+ motif collection by using `memelite`'s TomTom implementation.  
The resulting similarity matrix will be log-transformed and negated before performing clustering.  

In [8]:
sim_matrix = tm.pp.calculate_motif_similarity(
    seqlets_matrices,
    motif_collection,
    chunk_size=50000,  # you can supply a seqlet chunk_size if you have memory constraints
    n_nearest=100,  # you can limit the number of nearest motif similarities to store per seqlet to reduce memory usage
)

Processing chunks:   0%|          | 0/14 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 14/14 [26:24<00:00, 113.17s/it]


In [9]:
print(sim_matrix.shape)  # (n_seqlets, n_motifs)

(679653, 3989)


We can store the data objects we have processed so far together in a single Anndata object.  
This anndata object will become the input for all our `tfmindi.tl` tooling and `tfmindi.pl` plotting functions.  
The motif annotations etc are not mandatory but can be used to create visualisations later.  

In [10]:
adata = tm.pp.create_seqlet_adata(
    sim_matrix,  # mandatory
    seqlets_df,  # mandatory
    seqlet_matrices=seqlets_matrices,
    oh_sequences=oh,
    contrib_scores=contrib,
    motif_collection=motif_collection,
    motif_annotations=motif_annotations,
    motif_to_dbd=motif_to_db,
)
adata

AnnData object with n_obs × n_vars = 679653 × 3989
    obs: 'example_idx', 'start', 'end', 'attribution', 'p-value', 'seqlet_matrix', 'seqlet_oh', 'example_oh_idx', 'example_contrib_idx'
    var: 'motif_ppm', 'Direct_annot', 'Motif_similarity_annot', 'Orthology_annot', 'Motif_similarity_and_Orthology_annot', 'dbd'
    uns: 'unique_examples'

In [11]:
# optional: let's also add a "cell_type" column to the adata.obs based on our classes and example_idx (the region's idx)
adata.obs["cell_type"] = adata.obs["example_idx"].map(region_id_to_ct_map).astype("category")

In [12]:
adata.obs.head(3)

,example_idx,start,end,attribution,p-value,seqlet_matrix,seqlet_oh,example_oh_idx,example_contrib_idx,cell_type
0,18770,1052,1065,22.867273,2.963802e-15,"[[-0.38142416, -0.21325767, 0.56363165, -0.447...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0,0,Oligo
1,19895,1103,1134,42.865262,2.971223e-15,"[[0.9126998, 0.572017, 0.33544934, 0.30004725,...","[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,1,Oligo
2,19813,1045,1059,13.644972,4.303625e-15,"[[0.18059267, -0.007956118, -0.34745765, -0.09...","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...",2,2,Oligo


In [13]:
adata.var.head(3)

,motif_ppm,Direct_annot,Motif_similarity_annot,Orthology_annot,Motif_similarity_and_Orthology_annot,dbd
hocomoco__CENPB_HUMAN.H11MO.0.D,"[[0.04821992, 0.1383506, 0.012167643, 0.000450...",CENPB,NaN,NaN,NaN,CENPB
hocomoco__CPEB1_HUMAN.H11MO.0.D,"[[0.1636162, 0.0005546312, 0.0049916804, 0.000...",CPEB1,"ZNF287, ZNF362",NaN,NaN,C2H2 ZF
neph__UW.Motif.0049,"[[0.870051, 0.059264, 0.038039, 0.069257, 0.94...",NaN,"POU2F1, E2F1, YY1, PAX4, POU2F2",NaN,"NANOG, NANOGP8, SOX2",Homeodomain


## Saving our preprocesssed data

Let's save our anndata so we don't have to run these analyses again.  
We can't use standard Anndata functionality for this, since we're storing numpy arrays in both our .obs and .var. This is a logical way to structure our data, but is not allowed by Anndata when saving and loading.  
Therefore, we have our own I/O functions that are simple wrappers around Anndata's functions (wherein the numpy arrays are moved to and back from .uns, which does allow for arrays).  
You can save and load anndatas with {func}`~tfmindi.save_h5ad` and {func}`~tfmindi.load_h5ad`.
Be aware that these files can become large.

In [14]:
tm.save_h5ad(adata, DATA_DIR + "seqlets.h5ad")